In [1]:
import pandas as pd
import google.generativeai as genai
import os
import json
from copy import deepcopy
from dotenv import load_dotenv 

load_dotenv()

True

In [2]:
api_key = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=api_key)

In [3]:
model_name = os.getenv("MODEL_NAME", "gemini-1.5-flash")
model = genai.GenerativeModel(model_name)

In [69]:
def get_chatbot_response(messages):
    input_messages = []
    for message in messages:
        input_messages.append({"role": message["role"], "parts": message["parts"]})
    response = model.generate_content(messages)  
    
    return response

In [6]:
apriori_recommendation_path = 'api/recommendation_objects/apriori_recommendation.json'
popular_recommendation_path = 'api/recommendation_objects/popularity_recommendation.csv'

In [7]:
with open(apriori_recommendation_path, 'r') as file:
            apriori_recommendations = json.load(file)

In [8]:
popular_recommendations = pd.read_csv(popular_recommendation_path)
products = popular_recommendations['product'].tolist()
product_categories = popular_recommendations['product_category'].tolist()

In [9]:
popular_recommendations 


,product,product_category,number_of_transactions
0,Almond Croissant,Bakery,316
1,Cappuccino,Coffee,645
2,Carmel syrup,Flavours,486
3,Chocolate Chip Biscotti,Bakery,315
4,Chocolate Croissant,Bakery,601
5,Chocolate syrup,Flavours,506
6,Cranberry Scone,Bakery,308
7,Croissant,Bakery,330
8,Dark chocolate,Packaged Chocolate,19
9,Espresso shot,Coffee,620


In [10]:
products

['Almond Croissant',
 'Cappuccino',
 'Carmel syrup',
 'Chocolate Chip Biscotti',
 'Chocolate Croissant',
 'Chocolate syrup',
 'Cranberry Scone',
 'Croissant',
 'Dark chocolate',
 'Espresso shot',
 'Ginger Biscotti',
 'Ginger Scone',
 'Hazelnut Biscotti',
 'Hazelnut syrup',
 'Jumbo Savory Scone',
 'Latte',
 'Oatmeal Scone',
 'Sugar Free Vanilla syrup']

In [12]:
product_categories

['Bakery',
 'Coffee',
 'Flavours',
 'Bakery',
 'Bakery',
 'Flavours',
 'Bakery',
 'Bakery',
 'Packaged Chocolate',
 'Coffee',
 'Bakery',
 'Bakery',
 'Bakery',
 'Flavours',
 'Bakery',
 'Coffee',
 'Bakery',
 'Flavours']

In [13]:
def get_popular_recommendation(product_categories=None,top_k=5):
        recommendations_df = popular_recommendations
        
        if type(product_categories) == str:
            product_categories = [product_categories]

        if product_categories is not None:
            recommendations_df = popular_recommendations[popular_recommendations['product_category'].isin(product_categories)]
        recommendations_df = recommendations_df.sort_values(by='number_of_transactions',ascending=False)
        
        if recommendations_df.shape[0] == 0:
            return []

        recommendations = recommendations_df['product'].tolist()[:top_k]
        return recommendations

In [15]:
get_popular_recommendation()

['Cappuccino',
 'Latte',
 'Espresso shot',
 'Chocolate Croissant',
 'Sugar Free Vanilla syrup']

In [16]:
get_popular_recommendation('Bakery')

['Chocolate Croissant',
 'Ginger Scone',
 'Croissant',
 'Jumbo Savory Scone',
 'Almond Croissant']

In [40]:
apriori_recommendations

{'Carmel syrup': [{'product': 'Cappuccino',
   'product_categories': 'Coffee',
   'confidence': 0.3978021978021978},
  {'product': 'Latte',
   'product_categories': 'Coffee',
   'confidence': 0.37587006960556846},
  {'product': 'Espresso shot',
   'product_categories': 'Coffee',
   'confidence': 0.37244897959183676},
  {'product': 'Sugar Free Vanilla syrup',
   'product_categories': 'Flavours',
   'confidence': 0.32671480144404336},
  {'product': 'Chocolate syrup',
   'product_categories': 'Flavours',
   'confidence': 0.3068592057761733},
  {'product': 'Carmel syrup',
   'product_categories': 'Flavours',
   'confidence': 0.29347826086956524},
  {'product': 'Hazelnut syrup',
   'product_categories': 'Flavours',
   'confidence': 0.26739926739926745},
  {'product': 'Chocolate Croissant',
   'product_categories': 'Bakery',
   'confidence': 0.18478260869565216},
  {'product': 'Croissant',
   'product_categories': 'Bakery',
   'confidence': 0.17934782608695654}],
 'Cappuccino': [{'product': 

In [62]:
def get_apriori_recommendation(products,top_k=7):
        recommendation_list = []
        for product in products:
            if product in apriori_recommendations:
                  recommendation_list = recommendation_list + apriori_recommendations[product]

        # Sort recommendation list by "confidence"
        recommendation_list = sorted(recommendation_list,key=lambda x: x['confidence'],reverse=True)
        recommendations = []
        recommendations_per_category = {}
        for recommendation in recommendation_list:
            # If Duplicated recommendations then skip
            if recommendation in recommendations:
                continue 
            # Limit 2 recommendations per category
            product_category = recommendation['product_categories']
            if product_category not in recommendations_per_category:
                recommendations_per_category[product_category] = 0
            
            if recommendations_per_category[product_category] >= 2:
                continue

            recommendations_per_category[product_category]+=1

            # Add recommendation
            recommendations.append(recommendation['product'])

            if len(recommendations) >= top_k:
                break

        return recommendations 

    

In [65]:
get_apriori_recommendation(['Latte'])

['Cappuccino',
 'Latte',
 'Sugar Free Vanilla syrup',
 'Chocolate syrup',
 'Chocolate Croissant',
 'Croissant']

In [68]:
products = popular_recommendations['product'].tolist()
product_categories = popular_recommendations['product_category'].tolist()
products , product_categories

(['Almond Croissant',
  'Cappuccino',
  'Carmel syrup',
  'Chocolate Chip Biscotti',
  'Chocolate Croissant',
  'Chocolate syrup',
  'Cranberry Scone',
  'Croissant',
  'Dark chocolate',
  'Espresso shot',
  'Ginger Biscotti',
  'Ginger Scone',
  'Hazelnut Biscotti',
  'Hazelnut syrup',
  'Jumbo Savory Scone',
  'Latte',
  'Oatmeal Scone',
  'Sugar Free Vanilla syrup'],
 ['Bakery',
  'Coffee',
  'Flavours',
  'Bakery',
  'Bakery',
  'Flavours',
  'Bakery',
  'Bakery',
  'Packaged Chocolate',
  'Coffee',
  'Bakery',
  'Bakery',
  'Bakery',
  'Flavours',
  'Bakery',
  'Coffee',
  'Bakery',
  'Flavours'])

In [86]:
messages = []
system_prompt = """ You are a helpful AI assistant for a coffee shop application which serves drinks and pastries. We have 3 types of recommendations:

        1. Apriori Recommendations: These are recommendations based on the user's order history. We recommend items that are frequently bought together with the items in the user's order.
        2. Popular Recommendations: These are recommendations based on the popularity of items in the coffee shop. We recommend items that are popular among customers.
        3. Popular Recommendations by Category: Here the user asks to recommend them product in a category. Like what coffee do you recommend me to get?. We recommend items that are popular in the category of the user's requested category.
        
        Here is the list of items in the coffee shop:
        """+ ",".join(products) + """
        Here is the list of Categories we have in the coffee shop:
        """ + ",".join(product_categories) + """

        Your task is to determine which type of recommendation to provide based on the user's message.

        Your output should be in a structured json format like so. Each key is a string and each value is a string. Make sure to follow the format exactly:
        {
        "chain of thought": Write down your critical thinking about what type of recommendation is this input relevant to.
        "recommendation_type": "apriori" or "popular" or "popular by category". Pick one of those and only write the word.
        "parameters": This is a  python list. It's either a list of of items for apriori recommendations or a list of categories for popular by category recommendations. Leave it empty for popular recommendations. Make sure to use the exact strings from the list of items and categories above.
        }
        """
prompt = input("User: ")
messages.append({"role": "user", "parts": prompt})
input_messages = [{"role": "model", "parts": system_prompt}] + messages

chatbot_output =get_chatbot_response(input_messages).text
chatbot_output

'```json\n{\n  "chain of thought": "The user is asking for a recommendation within the \\"Bakery\\" category.  This is a request for a popular item within a specific category.",\n  "recommendation_type": "popular by category",\n  "parameters": ["Bakery"]\n}\n```\n'

In [90]:
def postprocess_classfication(output):
        cleaned_output = "\n".join(output.splitlines()[1:]).strip('`')
        output = json.loads(cleaned_output)

        dict_output = {
            "recommendation_type": output['recommendation_type'],
            "parameters": output['parameters'],
        }
        return dict_output

In [91]:
output = postprocess_classfication(chatbot_output)
output

{'recommendation_type': 'popular by category', 'parameters': ['Bakery']}

In [94]:
order = [
    {'item': 'Latte'},
    {'item': 'Croissant'}
]
products = []
for product in order:
    products.append(product['item'])

recommendations = get_apriori_recommendation(products)
recommendations_str = ", ".join(recommendations)

system_prompt = f"""
        You are a helpful AI assistant for a coffee shop application which serves drinks and pastries.
        your task is to recommend items to the user based on their order.

        I will provide what items you should recommend to the user based on their order in the user message. 
        """

prompt = f"""
{messages[-1]['parts']}

        Please recommend me those items exactly: {recommendations_str}
        """

messages[-1]['parts'] = prompt
input_messages = [{"role": "model", "parts": system_prompt}] + messages[-3:]

In [97]:
chatbot_output =get_chatbot_response(input_messages).text
chatbot_output

"Okay, you've ordered two Cappuccinos, two Sugar Free Vanilla syrups, and two Chocolate Croissants.  Since you seem to enjoy chocolate and coffee, I'd recommend trying our:\n\n* **Almond Croissant:**  A delicious alternative to the chocolate croissant, offering a different flavor profile.\n* **Pain au Chocolat:**  Similar to the chocolate croissant, but with a slightly different pastry and chocolate arrangement.  You might appreciate trying this variation.\n\nIf you're looking for something sweeter to complement your coffee, we also have:\n\n* **Chocolate Brownie:** A rich and decadent treat to enjoy with your Cappuccino.\n\n\nFor a lighter option:\n\n* **Plain Croissant:**  A classic choice to balance the richness of the chocolate croissants and coffee.\n\nLet me know if you'd like more suggestions!\n"

In [100]:
def postprocess(output):
        output = {
            "role": "assistant",
            "parts": output,
            "memory": {"agent":"recommendation_agent"
                      }
        }
        return output


In [99]:
output = postprocess(chatbot_output)
output

{'role': 'assistant',
 'content': "Okay, you've ordered two Cappuccinos, two Sugar Free Vanilla syrups, and two Chocolate Croissants.  Since you seem to enjoy chocolate and coffee, I'd recommend trying our:\n\n* **Almond Croissant:**  A delicious alternative to the chocolate croissant, offering a different flavor profile.\n* **Pain au Chocolat:**  Similar to the chocolate croissant, but with a slightly different pastry and chocolate arrangement.  You might appreciate trying this variation.\n\nIf you're looking for something sweeter to complement your coffee, we also have:\n\n* **Chocolate Brownie:** A rich and decadent treat to enjoy with your Cappuccino.\n\n\nFor a lighter option:\n\n* **Plain Croissant:**  A classic choice to balance the richness of the chocolate croissants and coffee.\n\nLet me know if you'd like more suggestions!\n",
 'memory': {'agent': 'recommendation_agent'}}